# Case - Machine Learning Engineering


**O Problema**

O **Minhas Finanças** é um produto no PicPay para organizar e comparar os gastos financeiros dos nossos usuários. Desta forma, o time de MLOPS do PicPay tem o objetivo de disponibilizar uma base de dados tratada para ser analisada pelos Cientistas de dados.

O **objetivo** desse exercício é realizar o tratamento de dados para analise das das transações que foram categorizadas.

De forma objetiva, o seu output final deve ser uma tabela conforme a imagem abaixo.


![DataFrame Esperado](resultado_esperado.png)


**Importante!**
  - Você deverá disponibilizar o código em um repositório no GitHub.
  - Recomendamos o uso de Pandas ou PySpark para tratamento dos dados.

**Ambiente de desenvolvimento**
  - Recomendamos o uso do Google Colab ou do Jupyter Notebook Localmente.

**Onde estão os dados?**
 - Dentro da pasta dataset existem dois arquivos `.csv` com os seguintes nomes: `bank_dim` e `transations`.
  

**Schema dos dados de entrada**

- **`transactions`:**
  - **transaction_id**: Chave de identificação da transação
  - **user_id**: Chave de identificação do usuário no sistema.
  - **transaction_name_raw**: Nome da transação em seu formato original
  - **transaction_name_treated**: Nome da transação com tratamento aplicado
  - **transaction_amount**: Valor da transação
  - **bank_id**: Chave de Identificação do banco de origem.
  - **year**: Ano da transação
  - **month**: Mês da transação
  - **day**: Dia da transação

- **`bank_dim`**
  - **bank_name**: Nome do banco.
  - **bank_id**: Identificador do banco.


In [56]:
import pandas as pd
import numpy as np

In [57]:
transactions = pd.read_csv("dataset/transactions.csv")
bank_dim = pd.read_csv("dataset/bank_dim.csv")

In [58]:
transactions.isnull().any().any()

False

In [59]:
bank_dim.isnull().any().any()

False

In [60]:
transactions_bank = transactions.merge(bank_dim, on="bank_id", how="left")

In [61]:
transactions_bank.isnull().any().any()

False

TRATAR COLUNAS

In [62]:
transactions_bank['transaction_name_treated'] = transactions_bank['transaction_name_treated'].apply(lambda x: x.upper())
transactions_bank['transaction_amount'] = transactions_bank['transaction_amount'].apply(lambda x: x.replace(",", ".")).astype("float64")

types = {
    "transaction_id": str,
    "user_id": str,
    "year": str,
    "month": str,
    "day": str,
    "bank_id": str,
}
transactions_bank = transactions_bank.astype(types)

CRIAR COLUNA YEAR-MONTH

In [63]:
transactions_bank['year_month'] = transactions_bank["year"].astype("str") + "-" + transactions_bank["month"].astype("str")

# VALIDAR O QUE EU QUERIA

In [64]:
# colum para analisar
column = "AUTO POSTO"

In [65]:
filtro1 = transactions_bank['user_id'] == "1005"
filtro2 = transactions_bank['year'] == "2022"
filtro3 = transactions_bank['month'] == "10"
filtro4 = transactions_bank['transaction_name_treated'] == column


transactions_bank[filtro1 & filtro2 & filtro3 & filtro4]

,transaction_id,user_id,transaction_name_raw,transaction_name_treated,transaction_amount,year,month,day,bank_id,bank_name,year_month
109,995925136,1005,RSCSS-AUTO POSTO -07/10,AUTO POSTO,-100.0,2022,10,24,6001,Nubank,2022-10


In [66]:
transactions_bank[filtro1 & filtro2 & filtro3 & filtro4]['transaction_amount'].sum()

-100.0

# criando a tabela desejada

In [54]:
columns_no_need = ["transaction_id", "transaction_name_raw", "year", "month", "day", "bank_id"]
transactions_bank = transactions_bank.drop(columns_no_need, axis=1)

In [25]:
df = transactions_bank.pivot_table(
    index=["user_id", 'year_month', 'bank_name'],
    columns="transaction_name_treated",
    values="transaction_amount",
    aggfunc="sum",
).reset_index()

In [31]:
df

transaction_name_treated,user_id,year_month,bank_name,AUTO POSTO,DROGARIA,FARMACIA,IFOOD,LUIZA,MERCADO,NETFLIX,POSTO,SPOTIFY,SUBWAY,UBER
0,1005,2022-10,Nubank,-100.00,NaN,NaN,-113.16,NaN,-923.02,NaN,NaN,NaN,-113.5,NaN
1,1005,2022-11,Nubank,-356.60,NaN,NaN,-97.17,NaN,-33.00,NaN,-65.00,NaN,NaN,-23.33
2,1005,2022-12,Nubank,NaN,-29.99,NaN,NaN,NaN,-52.00,NaN,NaN,NaN,NaN,-210.00
3,1005,2022-6,Nubank,-13.99,-4.49,NaN,-56.00,-290.92,-276.94,NaN,-40.00,-9.9,NaN,-12.59
4,1005,2022-7,Nubank,NaN,NaN,NaN,NaN,NaN,-94.43,NaN,-90.00,NaN,NaN,-78.49
5,1005,2022-8,Nubank,NaN,-253.63,-309.80,NaN,NaN,NaN,NaN,-20.55,-24.9,NaN,NaN
6,1005,2022-9,Nubank,NaN,NaN,NaN,NaN,NaN,-778.32,-105.7,NaN,-19.9,NaN,NaN
7,1006,2022-10,Bradesco,NaN,NaN,NaN,-73.00,NaN,-1165.89,NaN,NaN,NaN,NaN,NaN
8,1006,2022-11,Bradesco,-631.02,NaN,NaN,-332.92,NaN,NaN,NaN,-406.34,NaN,NaN,-63.55
9,1006,2022-12,Bradesco,NaN,NaN,NaN,NaN,NaN,-480.02,NaN,NaN,NaN,NaN,-188.56
